In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Define the path to your main directory containing the subfolders
base_dir = '/content/drive/My Drive/kidney'  # Adjust the folder name if necessary

# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% of data will be used for validation

# Training data generator
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 6991 images belonging to 3 classes.
Found 1746 images belonging to 3 classes.


In [ ]:
import os

# Define the base directory path
base_dir = '/content/drive/MyDrive/kidney'  # Adjust the folder name if necessary

categories = ['Normal', 'Tumor', 'Stone']

# Count the number of images in each category
for category in categories:
    folder_path = os.path.join(base_dir, category)
    # Check if the directory exists
    if os.path.exists(folder_path):
        num_images = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
        print(f'Number of images in {category} folder: {num_images}')
    else:
        print(f'Directory not found: {folder_path}')

Number of images in Normal folder: 5077
Number of images in Tumor folder: 2283
Number of images in Stone folder: 1377


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: normal, tumor, stone
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=3  # You can adjust the number of epochs
)


Epoch 1/3
218/218 [==============================] - 3143s 14s/step - loss: 0.5897 - accuracy: 0.7626 - val_loss: 0.7295 - val_accuracy: 0.7274
Epoch 2/3
218/218 [==============================] - 125s 575ms/step - loss: 0.1708 - accuracy: 0.9369 - val_loss: 0.7485 - val_accuracy: 0.8119
Epoch 3/3
218/218 [==============================] - 104s 478ms/step - loss: 0.0764 - accuracy: 0.9736 - val_loss: 0.6188 - val_accuracy: 0.8171


In [ ]:
model.save('tumor_detection_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


55/55 [==============================] - 20s 372ms/step - loss: 0.6180 - accuracy: 0.8139
Validation Accuracy: 81.39%


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving renal-tumour.jpg to renal-tumour.jpg


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Load the trained model
model = tf.keras.models.load_model('tumor_detection_model.h5')

# Define the class labels
class_labels = ['Normal', 'Stone', 'Tumor']

def load_and_preprocess_image(img_path):
    """
    Load and preprocess the image to the required input shape.
    """
    img = image.load_img(img_path, target_size=(150, 150))  # Image size should match the input size of the model
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the model's input shape
    img_array /= 255.0  # Normalize the image to match the preprocessing done during training
    return img_array

def predict_image(img_path):
    """
    Predict whether the input image is normal, stone, or tumor.
    """
    img_array = load_and_preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_label = class_labels[predicted_class[0]]
    confidence = np.max(predictions)

    return predicted_label, confidence

# Example usage
image_path = '/content/drive/MyDrive/kidney/Tumor/Tumor- (1052).jpg'  # Replace with the path to your image
predicted_label, confidence = predict_image(image_path)

print(f'Predicted: {predicted_label} with confidence {confidence:.2f}')


1/1 [==============================] - 0s 69ms/step
Predicted: Tumor with confidence 0.99
